In [24]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import calendar

In [25]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

# Import 2021 Picks

In [26]:
df = pd.read_csv('backend\\data\\predictions\\2021_nn.csv')
df['date'] = pd.to_datetime(df['date'])
df['y'] = np.where(df['y'], df['home'], df['away'])
df['y_pred'] = np.where(df['y_pred'], df['home'], df['away'])
df.dropna(inplace=True)
df

,date,home,away,week,season,y,ml_h,ml_a,y_pred,y_prob_a,...,prob_a,prob_h,h_diff,a_diff,pick_diff,pick_fav,pick_odds,risk_correct,risk_unit,risk_profit
0,2021-09-30,Cincinnati Bengals,Jacksonville Jaguars,4,2021,Cincinnati Bengals,-350,270,Cincinnati Bengals,0.026688,...,0.270270,0.777778,0.195534,-0.243582,0.195534,1,-350,1,2.00,0.571429
1,2021-10-03,Buffalo Bills,Houston Texans,4,2021,Buffalo Bills,-2200,1100,Buffalo Bills,0.158978,...,0.083333,0.956522,-0.115499,0.075644,-0.115499,1,-2200,1,0.75,0.034091
2,2021-10-03,Denver Broncos,Baltimore Ravens,4,2021,Baltimore Ravens,-120,100,Baltimore Ravens,0.622620,...,0.500000,0.545455,-0.168075,0.122620,0.122620,0,100,1,1.00,1.000000
3,2021-10-03,New York Jets,Tennessee Titans,4,2021,New York Jets,205,-255,New York Jets,0.476896,...,0.718310,0.327869,0.195235,-0.241413,0.195235,0,205,1,2.00,4.100000
4,2021-10-07,Seattle Seahawks,Los Angeles Rams,5,2021,Los Angeles Rams,130,-150,Los Angeles Rams,0.585027,...,0.600000,0.434783,-0.019809,-0.014973,-0.014973,1,-150,1,0.10,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2022-01-16,Dallas Cowboys,San Francisco 49ers,19,2021,San Francisco 49ers,-170,150,Dallas Cowboys,0.249186,...,0.400000,0.629630,0.121184,-0.150814,0.121184,1,-170,0,1.00,-1.000000
102,2022-01-16,Kansas City Chiefs,Pittsburgh Steelers,19,2021,Kansas City Chiefs,-600,435,Kansas City Chiefs,0.000357,...,0.186916,0.857143,0.142500,-0.186559,0.142500,1,-600,1,1.00,0.166667
103,2022-01-16,Tampa Bay Buccaneers,Philadelphia Eagles,19,2021,Tampa Bay Buccaneers,-335,260,Tampa Bay Buccaneers,0.071505,...,0.277778,0.770115,0.158380,-0.206273,0.158380,1,-335,1,1.50,0.447761
104,2022-01-30,Kansas City Chiefs,Cincinnati Bengals,21,2021,Cincinnati Bengals,-350,270,Kansas City Chiefs,0.278689,...,0.270270,0.777778,-0.056467,0.008419,-0.056467,1,-350,0,0.25,-0.250000


In [27]:
week = df[df['week'] == 9]
for index, row in week.iterrows():
    ml_a = '+{odds}'.format(odds=row['ml_a']) if row['ml_a'] > 0 else row['ml_a']
    ml_h = '+{odds}'.format(odds=row['ml_h']) if row['ml_h'] > 0 else row['ml_h']
    win = 'W' if row['y_pred'] == row['y'] else 'L'
    print(f"{row['away']} ({ml_a}) @ {row['home']} ({ml_h})")
    print(f"\t {row['y_pred']} {row['risk_unit']}u {win}")

New York Jets (+340) @ Indianapolis Colts (-450)
	 Indianapolis Colts 1.0u W
Denver Broncos (+340) @ Dallas Cowboys (-450)
	 Dallas Cowboys 1.5u L
Buffalo Bills (-1125) @ Jacksonville Jaguars (+700)
	 Buffalo Bills 0.25u L
Green Bay Packers (+260) @ Kansas City Chiefs (-335)
	 Kansas City Chiefs 0.1u W
Houston Texans (+160) @ Miami Dolphins (-190)
	 Miami Dolphins 0.1u W
Los Angeles Chargers (-110) @ Philadelphia Eagles (-110)
	 Philadelphia Eagles 1.5u L
Chicago Bears (+240) @ Pittsburgh Steelers (-305)
	 Pittsburgh Steelers 1.0u W


## Betting Analysis 

In [28]:
# Per Unit
per_unit = df.groupby(['risk_unit']).aggregate({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_unit['perc'] = (per_unit[('risk_correct', 'sum')] / per_unit[('risk_correct', 'count')] * 100).apply(round)
per_unit

risk_profit risk_correct       perc
                  sum          sum count     
risk_unit                                    
0.10         0.170213           12    15   80
0.25        -0.183631            7    11   64
0.50        -0.775604            5     8   62
0.75         3.169313           18    20   90
1.00        -0.947265           14    19   74
1.50        -5.933872           11    19   58
2.00         5.495326           11    14   79

In [29]:
# Per Fav
# Per Unit
per_fav = df.groupby(['pick_fav']).aggregate({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_fav['perc'] = (per_fav[('risk_correct', 'sum')] / per_fav[('risk_correct', 'count')] * 100).apply(round)
per_fav

risk_profit risk_correct       perc
                 sum          sum count     
pick_fav                                    
0            5.63750            7    12   58
1           -4.64302           71    94   76

In [30]:
df[df['pick_fav'] == 1][['date', 'home', 'ml_h', 'away', 'ml_a', 'y', 'y_pred', 'pick_diff', 'risk_unit', 'risk_profit']]

,date,home,ml_h,away,ml_a,y,y_pred,pick_diff,risk_unit,risk_profit
0,2021-09-30,Cincinnati Bengals,-350,Jacksonville Jaguars,270,Cincinnati Bengals,Cincinnati Bengals,0.195534,2.00,0.571429
1,2021-10-03,Buffalo Bills,-2200,Houston Texans,1100,Buffalo Bills,Buffalo Bills,-0.115499,0.75,0.034091
4,2021-10-07,Seattle Seahawks,130,Los Angeles Rams,-150,Los Angeles Rams,Los Angeles Rams,-0.014973,0.10,0.066667
6,2021-10-10,Dallas Cowboys,-320,New York Giants,250,Dallas Cowboys,Dallas Cowboys,0.199487,2.00,0.625000
7,2021-10-10,Jacksonville Jaguars,170,Tennessee Titans,-200,Tennessee Titans,Tennessee Titans,-0.141220,1.00,0.500000
...,...,...,...,...,...,...,...,...,...,...
101,2022-01-16,Dallas Cowboys,-170,San Francisco 49ers,150,San Francisco 49ers,Dallas Cowboys,0.121184,1.00,-1.000000
102,2022-01-16,Kansas City Chiefs,-600,Pittsburgh Steelers,435,Kansas City Chiefs,Kansas City Chiefs,0.142500,1.00,0.166667
103,2022-01-16,Tampa Bay Buccaneers,-335,Philadelphia Eagles,260,Tampa Bay Buccaneers,Tampa Bay Buccaneers,0.158380,1.50,0.447761
104,2022-01-30,Kansas City Chiefs,-350,Cincinnati Bengals,270,Cincinnati Bengals,Kansas City Chiefs,-0.056467,0.25,-0.250000


In [31]:
# Per Month
df['month'] = df['date'].dt.month
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])
per_month = df.groupby(['month']).aggregate({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_month['perc'] = (per_month[('risk_correct', 'sum')] / per_month[('risk_correct', 'count')] * 100).apply(round)
per_month.sort_values(by=[('risk_profit', 'sum'), ], ascending=False)

risk_profit risk_correct       perc
              sum          sum count     
month                                    
Dec      7.698880           27    32   84
Jan      1.146247           17    21   81
Oct      0.876189           20    28   71
Feb      0.789474            1     1  100
Sep      0.571429            1     1  100
Nov    -10.087739           12    23   52

In [32]:
# Per Week
per_week = df.groupby(['week']).aggregate({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_week['perc'] = (per_week[('risk_correct', 'sum')] / per_week[('risk_correct', 'count')] * 100).apply(round)
per_week.sort_values([('risk_profit', 'sum')])

risk_profit risk_correct       perc
             sum          sum count     
week                                    
10     -4.816092            3     7   43
9      -2.617427            4     7   57
11     -2.540031            2     4   50
6      -1.802273            3     4   75
7      -1.436553            5     7   71
8      -1.079113            3     6   50
15     -0.665635            7    10   70
12     -0.257046            2     4   50
21     -0.250000            0     1    0
19     -0.207001            3     4   75
5       0.202894            7     9   78
16      0.328416            4     5   80
17      0.444514            9    10   90
22      0.789474            1     1  100
18      1.158733            5     6   83
14      3.316249            8     8  100
13      4.719851            8     9   89
4       5.705519            4     4  100

In [33]:
df[df['week'] == 10][['date', 'home', 'ml_h', 'away', 'ml_a', 'y', 'y_pred', 'pick_diff', 'risk_unit', 'risk_profit']]

,date,home,ml_h,away,ml_a,y,y_pred,pick_diff,risk_unit,risk_profit
37,2021-11-11,Miami Dolphins,310,Baltimore Ravens,-410,Miami Dolphins,Baltimore Ravens,0.195174,2.00,-2.000000
38,2021-11-14,Arizona Cardinals,-310,Carolina Panthers,245,Carolina Panthers,Arizona Cardinals,0.030071,0.25,-0.250000
39,2021-11-14,Indianapolis Colts,-450,Jacksonville Jaguars,340,Indianapolis Colts,Indianapolis Colts,0.117002,0.75,0.166667
40,2021-11-14,Las Vegas Raiders,125,Kansas City Chiefs,-145,Kansas City Chiefs,Kansas City Chiefs,0.105153,0.75,0.517241
41,2021-11-14,Los Angeles Chargers,-180,Minnesota Vikings,155,Minnesota Vikings,Los Angeles Chargers,0.194848,2.00,-2.000000
42,2021-11-14,New York Jets,550,Buffalo Bills,-800,Buffalo Bills,Buffalo Bills,-0.197165,2.00,0.250000
43,2021-11-14,Washington Football Team,340,Tampa Bay Buccaneers,-450,Washington Football Team,Tampa Bay Buccaneers,0.171814,1.50,-1.500000


In [34]:
# Per Team
per_team = df.groupby(['y_pred']).aggregate({'risk_profit': 'sum', 'risk_correct': ['sum', 'count']})
per_team['perc'] = (per_team[('risk_correct', 'sum')] / per_team[('risk_correct', 'count')] * 100).apply(round)
per_team.sort_values(by=[('risk_profit', 'sum'), ], ascending=False)

risk_profit risk_correct       perc
                                 sum          sum count     
y_pred                                                      
New York Jets               4.100000            1     1  100
Los Angeles Rams            2.819224            7     7  100
Washington Football Team    2.100000            1     1  100
Chicago Bears               1.690323            2     2  100
Minnesota Vikings           0.813632            2     2  100
Kansas City Chiefs          0.723400            9    11   82
Atlanta Falcons             0.600000            1     1  100
Miami Dolphins              0.551619            3     3  100
Pittsburgh Steelers         0.327869            1     1  100
New York Giants             0.312500            1     1  100
San Francisco 49ers         0.270359            3     3  100
Green Bay Packers           0.125000            1     1  100
Tennessee Titans            0.030477            4     5   80
New England Patriots        0.022222            1     1  100
New Orleans Saints          0.005952            2     3   67
Cleveland Browns           -0.096154            1     2   50
Los Angeles Chargers       -0.118382            3     4   75
Indianapolis Colts         -0.352525            4     6   67
Denver Broncos             -0.458333            1     2   50
Cincinnati Bengals         -0.850000            2     4   50
Arizona Cardinals          -0.989583            1     3   33
Tampa Bay Buccaneers       -1.300503            8    11   73
Dallas Cowboys             -1.389719            3     5   60
Buffalo Bills              -1.459298            7     9   78
Philadelphia Eagles        -1.643806            5     8   62
Baltimore Ravens           -2.310382            3     6   50
Seattle Seahawks           -2.529412            1     3   33